In [66]:
import numpy as np
import pandas as pd
from pathlib import Path

import yfinance as yf
import ta
from tqdm import tqdm

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [22]:
should_load_new_data = True

# Explore what data we can get

In [5]:
stock = yf.Ticker("NVDA")

In [8]:
stock.history(period="1mo").tail()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2025-02-24 00:00:00-05:00,136.559998,138.589996,130.080002,130.279999,251381100,0.0,0.0
2025-02-25 00:00:00-05:00,129.979996,130.199997,124.440002,126.629997,271428700,0.0,0.0
2025-02-26 00:00:00-05:00,129.990005,133.729996,128.490005,131.279999,322553800,0.0,0.0
2025-02-27 00:00:00-05:00,135.000000,135.009995,120.010002,120.150002,443175800,0.0,0.0
2025-02-28 00:00:00-05:00,118.019997,125.089996,116.400002,124.919998,388160100,0.0,0.0


Everyday stock prices, include:
- Open, high, low, close: obvious
- Volume: Amount of stock exchanged
- Dividends: Paid on that date if exists, else 0
- Stock splits: Is when a company increases the number of shares outstanding by dividing its existing shares. The price of each share decreases, but the total shareholder value remains the same.

For example:
🔹 2-for-1 split: If you have 1 share of stock worth 200, after the split, you will have 2 shares worth 100 each.

So these are the columns we are gonna try to predict

In [5]:
stock.option_chain(stock.options[0]).calls.tail()

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
54,MSFT250307C00525000,2025-02-07 17:06:29+00:00,525.0,0.03,0.0,0.21,0.0,0.0,1.0,4,0.875001,False,REGULAR,USD
55,MSFT250307C00530000,2025-02-27 17:58:01+00:00,530.0,0.01,0.0,0.21,0.0,0.0,6.0,34,0.900392,False,REGULAR,USD
56,MSFT250307C00540000,2025-02-14 14:30:10+00:00,540.0,0.02,0.0,0.02,0.0,0.0,10.0,26,0.765627,False,REGULAR,USD
57,MSFT250307C00550000,2025-01-30 20:57:35+00:00,550.0,0.05,0.0,0.21,0.0,0.0,NaN,19,1.000005,False,REGULAR,USD
58,MSFT250307C00560000,2025-01-30 20:39:00+00:00,560.0,0.04,0.0,0.21,0.0,0.0,NaN,22,1.048833,False,REGULAR,USD


In [45]:
stock.balance_sheet.head()

,2025-01-31,2024-01-31,2023-01-31,2022-01-31,2021-01-31
Treasury Shares Number,NaN,NaN,NaN,NaN,13800000000.0
Ordinary Shares Number,24477000000.0,24640000000.0,24661365720.0,25060000000.0,NaN
Share Issued,24477000000.0,24640000000.0,24661365720.0,25060000000.0,NaN
Net Debt,NaN,2429000000.0,7564000000.0,8956000000.0,6116000000.0
Total Debt,10270000000.0,11056000000.0,12031000000.0,11831000000.0,NaN


In [47]:
stock.balance_sheet.shape

(86, 5)

In [8]:
stock.balance_sheet.index

Index(['Ordinary Shares Number', 'Share Issued', 'Net Debt', 'Total Debt',
       'Tangible Book Value', 'Invested Capital', 'Working Capital',
       'Net Tangible Assets', 'Capital Lease Obligations',
       'Common Stock Equity', 'Total Capitalization',
       'Total Equity Gross Minority Interest', 'Stockholders Equity',
       'Gains Losses Not Affecting Retained Earnings',
       'Other Equity Adjustments', 'Retained Earnings', 'Capital Stock',
       'Common Stock', 'Total Liabilities Net Minority Interest',
       'Total Non Current Liabilities Net Minority Interest',
       'Other Non Current Liabilities', 'Tradeand Other Payables Non Current',
       'Non Current Deferred Liabilities', 'Non Current Deferred Revenue',
       'Non Current Deferred Taxes Liabilities',
       'Long Term Debt And Capital Lease Obligation',
       'Long Term Capital Lease Obligation', 'Long Term Debt',
       'Current Liabilities', 'Other Current Liabilities',
       'Current Deferred Liabilities',

Those yearly report should be useful

In [50]:
stock.quarterly_income_stmt.head()

,2025-01-31,2024-10-31,2024-07-31,2024-04-30,2024-01-31,2023-10-31,2023-07-31
Tax Effect Of Unusual Items,0.0,0.0,0.0,0.0,0.0,NaN,NaN
Tax Rate For Calcs,0.123964,0.135,0.136,0.139,0.129094,NaN,NaN
Normalized EBITDA,25821000000.0,22855000000.0,19708000000.0,17753000000.0,14556000000.0,NaN,NaN
Total Unusual Items,NaN,NaN,NaN,NaN,0.0,0.0,0.0
Total Unusual Items Excluding Goodwill,NaN,NaN,NaN,NaN,0.0,0.0,0.0


In [51]:
stock.quarterly_income_stmt.shape

(43, 7)

In [11]:
stock.quarterly_income_stmt.index

Index(['Tax Effect Of Unusual Items', 'Tax Rate For Calcs',
       'Normalized EBITDA', 'Total Unusual Items',
       'Total Unusual Items Excluding Goodwill',
       'Net Income From Continuing Operation Net Minority Interest',
       'Reconciled Depreciation', 'Reconciled Cost Of Revenue', 'EBITDA',
       'EBIT', 'Net Interest Income', 'Interest Expense', 'Interest Income',
       'Normalized Income',
       'Net Income From Continuing And Discontinued Operation',
       'Total Expenses', 'Total Operating Income As Reported',
       'Diluted Average Shares', 'Basic Average Shares', 'Diluted EPS',
       'Basic EPS', 'Diluted NI Availto Com Stockholders',
       'Net Income Common Stockholders', 'Net Income',
       'Net Income Including Noncontrolling Interests',
       'Net Income Continuous Operations', 'Tax Provision', 'Pretax Income',
       'Other Income Expense', 'Other Non Operating Income Expenses',
       'Special Income Charges', 'Write Off', 'Gain On Sale Of Security',
  

In [12]:
stock.calendar

{'Dividend Date': datetime.date(2025, 3, 13),
 'Ex-Dividend Date': datetime.date(2025, 2, 20),
 'Earnings Date': [datetime.date(2025, 4, 24), datetime.date(2025, 4, 29)],
 'Earnings High': 3.56,
 'Earnings Low': 3.16,
 'Earnings Average': 3.22615,
 'Revenue High': 69897682310,
 'Revenue Low': 67140000000,
 'Revenue Average': 68534440260}

- Dividend Date: Ngày chi trả cổ tức tiếp theo.
- Ex-Dividend Date: Ngày giao dịch không hưởng quyền cổ tức.
- Earnings Date: Khoảng thời gian công ty dự kiến công bố báo cáo thu nhập: Đây là thời điểm quan trọng vì giá cổ phiếu thường biến động mạnh sau khi công ty công bố lợi nhuận.

- Earnings High: Dự báo EPS (Lợi nhuận trên mỗi cổ phiếu) cao nhất từ các nhà phân tích.
- Earnings Low: Dự báo EPS thấp nhất.
- Earnings Average: Trung bình dự báo EPS.

- Revenue High: Doanh thu dự kiến cao nhất.
- Revenue Low: Doanh thu dự kiến thấp nhất.
- Revenue Average: Trung bình dự báo doanh thu.

In [49]:
# stock.info

In [9]:
stock.analyst_price_targets

{'current': 396.99,
 'high': 650.0,
 'low': 420.0,
 'mean': 508.051,
 'median': 500.0}

- current: Giá hiện tại của cổ phiếu trên thị trường.
- high: Mức giá cao nhất mà một số nhà phân tích dự đoán cổ phiếu có thể đạt được.
- low: Mức giá thấp nhất mà một số nhà phân tích dự đoán.
- mean: Mức giá trung bình từ dự báo của nhiều nhà phân tích.
- median: Mức giá trung vị trong dự báo của các nhà phân tích.
6 - 12 tháng trong tương lai

In [13]:
stock.eps_trend

,current,7daysAgo,30daysAgo,60daysAgo,90daysAgo
period,,,,,
0q,3.21293,3.21472,3.16529,3.17325,3.17383
+1q,3.33131,3.33346,3.29005,3.29549,3.29340
0y,13.15656,13.15656,13.02874,13.04388,13.08535
+1y,15.07137,15.07137,15.06393,15.08038,15.10941


# Load data

## Company information

In [3]:
tickers = ["MSFT", "AAPL", "TSLA", "GOOGL", "AMZN", "NVDA"]

In [48]:
all_info_data = []

for ticker in tickers:
    stock = yf.Ticker(ticker)
    balance_sheet = stock.balance_sheet
    years = balance_sheet.columns = balance_sheet.columns.year
    if len(years) > 4:
        years = years[1:]                                                # Take 4 nearest years

    data = {"Ticker": ticker}
    for feature in balance_sheet.index:
        for year in years:
            data[f"{feature}_{max(years) - year}ya"] = balance_sheet.at[feature, year]

    df = pd.DataFrame([data])
    
    all_info_data.append(df)

final_info_df = pd.concat(all_info_data, ignore_index=True)
final_info_df.tail()

,Ticker,Ordinary Shares Number_0ya,Ordinary Shares Number_1ya,Ordinary Shares Number_2ya,Ordinary Shares Number_3ya,Share Issued_0ya,Share Issued_1ya,Share Issued_2ya,Share Issued_3ya,Net Debt_0ya,...,Employee Benefits_2ya,Employee Benefits_3ya,Non Current Prepaid Assets_0ya,Non Current Prepaid Assets_1ya,Non Current Prepaid Assets_2ya,Non Current Prepaid Assets_3ya,Non Current Accounts Receivable_0ya,Non Current Accounts Receivable_1ya,Non Current Accounts Receivable_2ya,Non Current Accounts Receivable_3ya
1,AAPL,1.555006e+10,1.594342e+10,1.642679e+10,NaN,1.555006e+10,1.594342e+10,1.642679e+10,NaN,8.112300e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TSLA,3.185000e+09,3.164000e+09,3.099000e+09,NaN,3.185000e+09,3.164000e+09,3.099000e+09,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GOOGL,1.246000e+10,1.284900e+10,1.324242e+10,NaN,1.246000e+10,1.284900e+10,1.324242e+10,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AMZN,1.038300e+10,1.024200e+10,1.018000e+10,NaN,1.089800e+10,1.075700e+10,1.064000e+10,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NVDA,2.464000e+10,2.466137e+10,2.506000e+10,NaN,2.464000e+10,2.466137e+10,2.506000e+10,NaN,2.429000e+09,...,NaN,33000000.0,2.822000e+09,3.376000e+09,3.509000e+09,NaN,NaN,NaN,NaN,NaN


## Time series

In [85]:
parquet_file = Path("../data/raw/stock_data_till_Feb.parquet")

if should_load_new_data:
    all_ts_data = []
    
    for ticker in tqdm(tickers, desc="Loading ts data"):
        stock = yf.Ticker(ticker)
        df = stock.history(period="1y")

        df.index = df.index.date
        
        if df.empty:
            print(f"Lỗi: Không có dữ liệu cho {ticker}")
            continue
        
        df["Ticker"] = ticker

        # Thêm các chỉ số kỹ thuật
        df["SMA_10"] = ta.trend.sma_indicator(df["Close"], window=10)
        df["SMA_30"] = ta.trend.sma_indicator(df["Close"], window=30)
        df["SMA_60"] = ta.trend.sma_indicator(df["Close"], window=60)
        
        df["RSI_10"] = ta.momentum.rsi(df["Close"], window=10)
        df["RSI_30"] = ta.momentum.rsi(df["Close"], window=30)
        df["RSI_60"] = ta.momentum.rsi(df["Close"], window=60)
        
        df["MACD"] = ta.trend.macd(df["Close"])
        df["MACD_signal"] = ta.trend.macd_signal(df["Close"])
        df["ATR_10"] = ta.volatility.average_true_range(df["High"], df["Low"], df["Close"], window=10)  # ATR (volatility)
        df["Bollinger_Upper"], df["Bollinger_Lower"] = ta.volatility.bollinger_hband(df["Close"]), ta.volatility.bollinger_lband(df["Close"])  # Bollinger Bands
        
        df["CCI_10"] = ta.trend.cci(df["High"], df["Low"], df["Close"], window=10)
        df["CCI_30"] = ta.trend.cci(df["High"], df["Low"], df["Close"], window=30)
        df["CCI_60"] = ta.trend.cci(df["High"], df["Low"], df["Close"], window=60)
        
        df["DX_10"] = ta.trend.dpo(df["Close"], window=10)
        df["DX_30"] = ta.trend.dpo(df["Close"], window=30)
        df["DX_60"] = ta.trend.dpo(df["Close"], window=60)
        
        df["ADX_10"] = ta.trend.adx(df["High"], df["Low"], df["Close"], window=10)
        df["ADX_30"] = ta.trend.adx(df["High"], df["Low"], df["Close"], window=30)
        df["ADX_60"] = ta.trend.adx(df["High"], df["Low"], df["Close"], window=60)

        df["Returns"] = df["Close"].pct_change()
        mean_return = df["Returns"].mean()
        std_return = df["Returns"].std()
        df["Turbulence"] = np.abs((df["Returns"] - mean_return) / std_return)

        # Thêm giá Open/High/Low/Close của 5 ngày trước
        for lag in range(1, 6):
            df[f"Open_lag_{lag}"] = df["Open"].shift(lag)
            df[f"High_lag_{lag}"] = df["High"].shift(lag)
            df[f"Low_lag_{lag}"] = df["Low"].shift(lag)
            df[f"Close_lag_{lag}"] = df["Close"].shift(lag)

        # Target
        df["next_close"] = df["Close"].shift(-1)
        df["next_returns"] = df["Returns"].shift(-1)
        df["increase_decrease"] = (df["next_close"] > df["Close"]).astype(int)

        # df.index = df.index.tz_convert(None)
        df = df.groupby("Ticker").apply(lambda x: x.iloc[60:-1])

        all_ts_data.append(df)
    
    final_df = pd.concat(all_ts_data)
    
    final_df.to_parquet(parquet_file, engine="pyarrow")
    
    print(f"Data saved to {parquet_file.name}")

    # should_load_new_data = False

/tmp/ipykernel_1387/2058782508.py:62: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("Ticker").apply(lambda x: x.iloc[60:-1])
/tmp/ipykernel_1387/2058782508.py:62: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("Ticker").apply(lambda x: x.iloc[60:-1])
/tmp/ipykernel_1387/2058782508.py:62: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior

Data saved to stock_data_till_Feb.parquet


In [86]:
ts_df = pd.read_parquet(parquet_file)
ts_df.shape

(1134, 53)

In [88]:
# ts_df[["Close", "Returns", "next_close", "next_returns"]]
ts_df.head()

Open        High         Low       Close    Volume  \
Ticker                                                                        
MSFT   2024-05-29  423.226650  428.446270  423.226650  426.686523  15517100   
       2024-05-30  421.844664  421.844664  411.842882  412.270416  28424800   
       2024-05-31  414.338373  414.338373  402.169213  412.727753  47995300   
       2024-06-03  413.125423  414.020209  406.553688  411.127045  17484700   
       2024-06-04  410.043372  414.030177  407.309286  413.662323  14348900   

                   Dividends  Stock Splits Ticker      SMA_10      SMA_30  \
Ticker                                                                      
MSFT   2024-05-29        0.0           0.0   MSFT  424.114474  409.986047   
       2024-05-30        0.0           0.0   MSFT  423.278342  410.104409   
       2024-05-31        0.0           0.0   MSFT  422.695734  410.488437   
       2024-06-03        0.0           0.0   MSFT  422.030606  410.989473   
       2024-06-04        0.0           0.0   MSFT  421.108972  411.514152   

                   ...  High_lag_4   Low_lag_4  Close_lag_4  Open_lag_5  \
Ticker             ...                                                    
MSFT   2024-05-29  ...  429.907795  424.658350   428.028717  424.360018   
       2024-05-30  ...  431.090866  422.958209   424.529053  427.601213   
       2024-05-31  ...  428.565562  421.954050   427.670776  430.464507   
       2024-06-03  ...  428.326971  424.131390   427.829865  424.717962   
       2024-06-04  ...  428.446270  423.226650   426.686523  427.143855   

                   High_lag_5   Low_lag_5  Close_lag_5  next_close  \
Ticker                                                               
MSFT   2024-05-29  430.464502  422.391495   426.557251  412.270416   
       2024-05-30  429.907795  424.658350   428.028717  412.727753   
       2024-05-31  431.090866  422.958209   424.529053  411.127045   
       2024-06-03  428.565562  421.954050   427.670776  413.662323   
       2024-06-04  428.326971  424.131390   427.829865  421.556366   

                   next_returns  increase_decrease  
Ticker                                              
MSFT   2024-05-29     -0.033786                  0  
       2024-05-30      0.001109                  1  
       2024-05-31     -0.003878                  0  
       2024-06-03      0.006167                  1  
       2024-06-04      0.019083                  1  

[5 rows x 53 columns]

# Helper function

In [5]:
def calculate_stats(data, columns):
    if isinstance(columns, str):
        columns = [columns]

    stats = []
    for col in columns:
        if data[col].dtype in ['object', 'category']:
            counts = data[col].value_counts(dropna=False, sort=False)
            percents = data[col].value_counts(normalize=True, dropna=False, sort=False) * 100
            formatted = counts.astype(str) + ' (' + percents.round(2).astype(str) + '%)'
            stats_col = pd.DataFrame({'count (%)': formatted})
            stats.append(stats_col)
        else:
            stats_col = data[col].describe().to_frame().transpose()
            stats_col['missing'] = data[col].isnull().sum()
            stats_col.index.name = col
            stats.append(stats_col)

    return pd.concat(stats, axis=0)

# Time series data

In [12]:
stock = yf.Ticker("MSFT")
df = stock.history(period="1mo")
df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2025-02-03 00:00:00-05:00,410.776349,414.578723,407.842230,410.097717,25679100,0.0,0.0
2025-02-04 00:00:00-05:00,411.864167,413.091716,408.920058,411.544800,20532100,0.0,0.0
2025-02-05 00:00:00-05:00,411.524861,413.001880,409.578751,412.462982,16316700,0.0,0.0
2025-02-06 00:00:00-05:00,413.171550,417.363157,413.171550,414.987915,16309800,0.0,0.0
2025-02-07 00:00:00-05:00,415.646597,417.812238,407.283362,408.930054,22886800,0.0,0.0


In [14]:
# Volume của ngày tiếp theo
df['vol_shift_minus_1'] = df['Volume'].shift(-1)
df.tail()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,vol_shift_minus_1
Date,,,,,,,,
2025-02-24 00:00:00-05:00,408.510010,409.369995,399.320007,404.000000,26443700,0.0,0.0,29387400.0
2025-02-25 00:00:00-05:00,401.100006,401.920013,396.700012,397.899994,29387400,0.0,0.0,19619000.0
2025-02-26 00:00:00-05:00,398.010010,403.600006,394.250000,399.730011,19619000,0.0,0.0,21127400.0
2025-02-27 00:00:00-05:00,401.269989,405.739990,392.170013,392.529999,21127400,0.0,0.0,32826500.0
2025-02-28 00:00:00-05:00,392.660004,397.630005,386.570007,396.989990,32826500,0.0,0.0,NaN


In [15]:
df['increase_decrease'] = (df['Volume'].shift(-1) > df['Volume']).astype(int)
df.tail()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,vol_shift_minus_1,increase_decrease
Date,,,,,,,,,
2025-02-24 00:00:00-05:00,408.510010,409.369995,399.320007,404.000000,26443700,0.0,0.0,29387400.0,1
2025-02-25 00:00:00-05:00,401.100006,401.920013,396.700012,397.899994,29387400,0.0,0.0,19619000.0,0
2025-02-26 00:00:00-05:00,398.010010,403.600006,394.250000,399.730011,19619000,0.0,0.0,21127400.0,1
2025-02-27 00:00:00-05:00,401.269989,405.739990,392.170013,392.529999,21127400,0.0,0.0,32826500.0,1
2025-02-28 00:00:00-05:00,392.660004,397.630005,386.570007,396.989990,32826500,0.0,0.0,NaN,0
